# Análisis y Estimación de Costos (Gold)  
## Dataknow – Estimación de Costos para Equipo de Construcción

---

Este notebook realiza el **análisis estadístico descriptivo** de los precios de las materias primas **X, Y y Z**, así como la **estimación de costos** de los equipos del proyecto, aplicando las **reglas de negocio definidas**
1. **Equipo 1:** Composición del Precio: 20% la materia prima **X** y el resto por la
materia prima **Y**
2. **Equipo 2:** El precio del equipo 2 está conformado por partes iguales de las
materias primas **X**, **Y** y **Z**.
---

### Alcance del análisis

- Estadística descriptiva:
  - Promedios, medianas, rangos y desviaciones estándar por materia prima.
- Análisis temporal:
  - Identificación de tendencias generales y posibles patrones estacionales.
- Visualización:
  - Series temporales de precios.
  - Histogramas de distribución por insumo.
- Estimación de costos:
  - Cálculo del costo del **Equipo 1** y **Equipo 2** según las reglas de negocio.

---

### Información del proyecto

- **Equipo:** Dataknow  
- **Data Scientist:** Juan Pablo Guerra Osorio


In [0]:
# Carga de las tablas traidas desde el UC 
gold_x = spark.table("dataknow_dll.construccion.silver_x")
gold_y = spark.table("dataknow_dll.construccion.silver_y")
gold_z = spark.table("dataknow_dll.construccion.silver_z")

## Analisis descriptivo bascio para cada tabla
---